In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc  ###计算roc和auc
import joblib

In [14]:
#构建随机森林模型，导入训练好的参数
path = "../RandomForestClassifiter.model"   #直接使用上层目录训练好的参数
# model = RandomForestClassifier(min_samples_leaf=97, n_estimators=1, random_state=50)
# model.fit(x_train,np.ravel(y_train))
# y_pred = model.predict(x_test)
model = joblib.load(path)

In [3]:

interaction_image=np.genfromtxt('../yellowModule_interaction_Image_data_1_1.csv',delimiter=',')
interaction_image=interaction_image.reshape((12098,1024))
print(interaction_image.shape)     #关联的特征
# 使用PCA降维
pca_dim = 256  # 设置PCA的目标维度
SEED=42
pca = PCA(n_components=pca_dim, random_state=SEED)
pca.fit(interaction_image)

(12098, 1024)


PCA(n_components=256, random_state=42)

In [15]:
key_genes=["UBE2C", "TPX2", "FOXM1", "E2F1", "CEP55", "NEK2", "CDCA8", "CDK1", "CCNB1","CKS2"]
key_gene_df=pd.DataFrame(columns=["node1","node2"])
print(key_genes)

['UBE2C', 'TPX2', 'FOXM1', 'E2F1', 'CEP55', 'NEK2', 'CDCA8', 'CDK1', 'CCNB1', 'CKS2']


In [16]:
for i in range(9) :
    k=i+1
    for  a in range(k,10):
         key_gene_df=key_gene_df._append({"node1":key_genes[i],"node2":key_genes[a]},ignore_index=True)

In [17]:
#读取表达矩阵
expression_df=pd.read_csv("../data/yellow_moduel_final_expression_matrix_protein_coding_genes.csv")  #这里是获取表达矩阵  222是简化版的表达矩阵，去掉了未包含基因的表达  #1
expression_df=expression_df.iloc[:,1:]   #处理后的表达矩阵  #1
gene_index=expression_df.columns   #这里将表达矩阵的基因索引单独存放在一个array中   #1
gene_index=np.array(gene_index)     #1
expression_matrix=expression_df.values  #这里将df形式的表达矩阵转为array，方便用numpy进行运算   #1
expression_matrix.shape

(1173, 800)

In [18]:
#将转变图像的代码封装成函数形式   #1
def transform_image(gene1_name,gene2_name,gene_list,expression_matrix):   #这里传入的gene_list是表达矩阵中基因的顺序(不能修改)，传入的expression_matrix是array类型的
    index_1=np.where(gene_list==gene1_name)#这里是为了得到具有相互作用的两个基因在基因表达矩阵中对应的索引，有了索引，才能得到该基因在所有样本中的表达情况
    index_2=np.where(gene_list==gene2_name)
    if(index_1[0].size>0 and index_2[0].size>0):
        gene_1_expression=expression_matrix[:,index_1].reshape(1173)   #这里是获取某一个相互关系的一个端点基因的表达情况   这里的1166是样本数，reshape是为了降低维度
       
        gene_2_expression=expression_matrix[:,index_2].reshape(1173)   #这里是获取某一个相互关系的第二个端点基因的表达情况
        #这里通过numpy.histogram2d方法进行转换，将两个基因的表达情况转变为图像
        H,x_edges,y_edges=np.histogram2d(gene_1_expression,gene_2_expression,bins=32)
        H=H.T
        print(H.sum())
        #H=(np.log10(H/1166+10**-4)+4)/4      #这里是对每一个基因对图像的值做类似降维的处理，防止值过大
        print(H.sum())
        #print(H)
        return H
    else :
        return 0 

In [19]:
gene_interaction=transform_image("UBE2C","TPX2",gene_index,expression_matrix)
gene_interaction=gene_interaction.reshape(-1,1024)

1173.0
1173.0


In [20]:
# 使用PCA降维
gene_interaction = pca.transform(gene_interaction)
print(gene_interaction.shape)

(1, 256)


In [21]:
key_gene_df["score"]=0
key_gene_df

,node1,node2,score
0,UBE2C,TPX2,0
1,UBE2C,FOXM1,0
2,UBE2C,E2F1,0
3,UBE2C,CEP55,0
4,UBE2C,NEK2,0
5,UBE2C,CDCA8,0
6,UBE2C,CDK1,0
7,UBE2C,CCNB1,0
8,UBE2C,CKS2,0
9,TPX2,FOXM1,0


In [22]:
for i in range(45):
    a=key_gene_df.iloc[i][0]
    b=key_gene_df.iloc[i][1]
    image=transform_image(a,b,gene_index,expression_matrix)
    image=image.reshape(-1,1024)
    image=pca.transform(image)
    pred = model.predict(image)
    print(type(pred))
    key_gene_df.iat[i,2]=pred

1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.0
<class 'numpy.ndarray'>
1173.0
1173.

In [23]:
key_gene_df

,node1,node2,score
0,UBE2C,TPX2,1
1,UBE2C,FOXM1,1
2,UBE2C,E2F1,1
3,UBE2C,CEP55,1
4,UBE2C,NEK2,1
5,UBE2C,CDCA8,1
6,UBE2C,CDK1,1
7,UBE2C,CCNB1,1
8,UBE2C,CKS2,0
9,TPX2,FOXM1,1


In [24]:
#将结果写入文件
key_gene_df.to_csv('20241211_RandomForest_KeyGene_regulatory_Network.csv',sep=',')